In [127]:
#I am setting this environment to practice and validate dataset and build something out of it, hopefully!!
#and for easier analaysis, ofcourse, VSCode is not Data Engineers friendly, HAHAHA!

In [128]:
import pandas as pd
import os
import datetime

In [129]:
#bulk Df is basically block deals happend by the client.
#These clients can be FII and DII. We will build something to identfy that later on.
#for now, I am trying to consolidate the client regime, how much of Indian stock they bought.

bulk_df=pd.read_csv('../staging/bulk.csv')

#Schema changes: I am changing the schema as it is not good way to keep spaces in between.
#let's create a standard of camel casing i.e Quantity Traded -> QuantityTraded.
#Also, Let's Fix the Date Standards. 25-Aug-2023 -> 2023-08-25

new_schema={'Symbol':'SecuritiesCode','Client Name':'ClientName','Quantity Traded':'QuantityTraded', 'Trade Price / Wght. Avg. Price':'MeanWghtPrice', 'Buy/Sell':'Transaction'}
bulk_df_cleaned=bulk_df.rename(columns=new_schema).drop(columns=['Security Name','Remarks']).drop_duplicates()
bulk_df_cleaned['Date']=pd.to_datetime(bulk_df_cleaned.Date).dt.strftime('%Y-%m-%d')

#Negating QuantityTraded for Sell Transaction type to get the net Transaction.
#Net QunatityTraded
# - Positive -> Buy
# - Negative -> Sell
def negate_quantityTraded(Transaction,QuantityTraded):
    if Transaction == "SELL":
        return -QuantityTraded
    else:
        return QuantityTraded

bulk_df_cleaned['QuantityTraded']=bulk_df_cleaned.apply(lambda x: negate_quantityTraded(x['Transaction'],x['QuantityTraded']),axis=1)
bulk_df_cleaned.sort_values(by=['ClientName'],ascending=False).head(20)

#Possible case:
#   -  Now, we don't know how much shares these Client posses. Anyways we are concerned with Change.
#   -  There could be case of Intraday Trading i.e Buyer will buy/sell or sell/buy in same day, net buy=0
#   -  long term i.e CNC buy.

,Date,SecuritiesCode,ClientName,Transaction,QuantityTraded,MeanWghtPrice
61,2023-08-25,GLOBALPET,YUGA STOCKS AND COMMODITIES PRIVATE LIMITED .,SELL,-51000,92.08
15,2023-08-25,GLOBALPET,YUGA STOCKS AND COMMODITIES PRIVATE LIMITED .,BUY,9000,93.03
54,2023-08-25,BAHETI,VINOD SOMANI,SELL,-61500,143.12
80,2023-08-25,SAHANA,VICTUS ENTERPRISE LLP,SELL,-256000,188.86
43,2023-08-25,VIVIDHA,VIBRANT SECURITIES PVT. LTD,BUY,4051166,1.19
95,2023-08-25,VIVIDHA,VIBRANT SECURITIES PVT. LTD,SELL,-4768342,1.19
85,2023-08-25,URBAN,VEENA RAJESH SHAH,SELL,-37200,139.92
39,2023-08-25,URBAN,VEENA RAJESH SHAH,BUY,37200,141.21
42,2023-08-25,VIVIDHA,VANRAJ DADBHAI KAHOR,BUY,2862669,1.13
97,2023-08-25,VIVIDHA,VANRAJ DADBHAI KAHOR,SELL,-1787719,1.15


In [130]:
#now that we have some cleaning done, let check the regime of Client.
#Regime of Client: Varied Stocks that single client has bought.
#PS: you will see so many reassignments, coz I hate using more variables.

ClientPortfolio=bulk_df_cleaned
tdf=ClientPortfolio.groupby(['ClientName','Date','SecuritiesCode'])[['QuantityTraded']].sum().reset_index()

#TradeType: Indicates type of Trade occurred depending on Net QuantityTraded.
#if Net QunatityTraded is 0 -> Intraday Trading else or else we will consider it long term.
def determine_trade_type(NetQunatityTrade) -> str :
    if NetQunatityTrade == 0:
        return "INTRADAY"
    elif NetQunatityTrade > 0:
        return "BUY"
    else :
        return "SELL"

tdf['TradeType']=tdf.apply(lambda x: determine_trade_type(x['QuantityTraded']), axis=1 )
tdf.to_csv("../data/ClientPortFolio.csv")

#I am debating wether to filter out Intraday TradeType for now or not?
#But Keeping might give us better insight on which Stocks are good for intraday

tdf.head(20)
#I know this will only give us the Client who bought most number of stocks in a day, which it bought that is still with bulk_df_cleaned
#I have alogorithm to store all the stocks bought in fixed lenght, will implement it later on. 
# Just for guess, it has to do with bit map. HAHA!


,ClientName,Date,SecuritiesCode,QuantityTraded,TradeType
0,A S CONFIN PRIVATE LIMITED,2023-08-25,DIL,0,INTRADAY
1,AAKRAYA RESEARCH LLP,2023-08-25,ATULAUTO,0,INTRADAY
2,AAKRAYA RESEARCH LLP,2023-08-25,PARAGMILK,0,INTRADAY
3,ASCENT INVESTMENT HOLDINGS PTE.LIMITED.,2023-08-25,AMBER,-1260552,SELL
4,BHAVESHKUMAR NATVARLAL SHETH,2023-08-25,GLOBALPET,0,INTRADAY
5,CHAUDHARY JAGDISH MADHAVLAL,2023-08-25,SAHANA,130000,BUY
6,DIKSHA GUPTA,2023-08-25,VIVIDHA,-2000000,SELL
7,DIVYA BHANDARI,2023-08-25,SONUINFRA,-45000,SELL
8,G G ENGINEERING LIMITED,2023-08-25,GISOLUTION,-833802,SELL
9,GOVERNMENT OF SINGAPORE,2023-08-25,AMBER,725239,BUY
